# Пиплметр

В данной работе проводится обработка результатов исследования, целью которого является проверка работы оборудования и регистрации респондентов на пиплметре (измерительное устройство для фиксации телесмотрения).

В рамках этой процедуры будет вычеслены следующие показатели:
- Процент ложноположительных ответов респондентов;
- Процент ложноотрицательных ответов респондентов;

И выделены ситуации заочной регистрации респондентом факта просмотра телевизора.


## Данные

Подключим библиотеки и загрузим файлы.

In [191]:
import pandas as pd
import numpy as np

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving src_OnOff.xlsx to src_OnOff (2).xlsx
Saving src_survey.xlsx to src_survey (2).xlsx
User uploaded file "src_OnOff (2).xlsx" with length 19563552 bytes
User uploaded file "src_survey (2).xlsx" with length 168023 bytes


Загрузим данные в pandas, где их будет удобно обработать.

In [192]:
opros = pd.read_excel('src_survey.xlsx') # опросы
pipl = pd.read_excel('src_OnOff.xlsx')   # регистрации с устройств

### Опросы

Для начала проанализирум файл с опросами.

In [193]:
opros.head(10)

,code,passport,button,tv state,unit,day,cs_start,cs_finish,see,display,Кол-во человек в семье 4+
0,258104,1204558,А,выключен,0,07.02.22,16:25:18,16:26:09,нет регистрации,нет в комнате,1
1,258106,1205697,Б,включён,0,07.02.22,16:50:18,16:53:17,нет регистрации,в комнате,3
2,258106,1205697,А,включён,0,07.02.22,16:50:18,16:53:17,нет регистрации,нет в комнате,3
3,258106,1205697,В,включён,0,07.02.22,16:50:18,16:53:17,нет регистрации,в комнате,3
4,258109,1206994,А,выключен,1,2022-02-08 00:00:00,03:05:31,03:06:58,нет регистрации,нет в комнате,1
5,258109,1206994,А,выключен,0,2022-02-08 00:00:00,03:05:31,03:06:58,нет регистрации,нет в комнате,1
6,258110,1208135,А,выключен,2,09.02.22,14:31:41,14:33:41,нет регистрации,нет в комнате,2
7,258110,1208135,А,включён,1,09.02.22,14:31:41,14:33:41,нет регистрации,нет в комнате,2
8,258110,1208135,Б,выключен,2,09.02.22,14:31:41,14:33:41,нет регистрации,нет в комнате,2
9,258110,1208135,Б,включён,1,09.02.22,14:31:41,14:33:41,нет регистрации,в комнате,2


Для удобсва переведём столбцы с временам к единому формату времени, учитывая полную дату в `sc_start` и `cs_finish`.

In [194]:
opros['cs_start'] = pd.to_datetime(opros['day'].astype(str) + \
                                   pd.Series(np.array([' '] * len(opros))) + \
                                   opros['cs_start'].astype(str), format='%d.%m.%y %H:%M:%S', errors="coerce"). \
                                   fillna(pd.to_datetime(opros['day'].astype(str) + \
                                          pd.Series(np.array([' '] * len(opros))) + \
                                          opros['cs_start'].astype(str), format='%Y-%m-%d 00:00:00 %H:%M:%S', errors="coerce"))
opros['cs_finish'] = pd.to_datetime(opros['day'].astype(str) + \
                                   pd.Series(np.array([' '] * len(opros))) + \
                                   opros['cs_finish'].astype(str), format='%d.%m.%y %H:%M:%S', errors="coerce"). \
                                   fillna(pd.to_datetime(opros['day'].astype(str) + \
                                          pd.Series(np.array([' '] * len(opros))) + \
                                          opros['cs_finish'].astype(str), format='%Y-%m-%d 00:00:00 %H:%M:%S', errors="coerce"))
opros['day'] = pd.to_datetime(opros['day'])

opros.head(10)

,code,passport,button,tv state,unit,day,cs_start,cs_finish,see,display,Кол-во человек в семье 4+
0,258104,1204558,А,выключен,0,2022-07-02,2022-02-07 16:25:18,2022-02-07 16:26:09,нет регистрации,нет в комнате,1
1,258106,1205697,Б,включён,0,2022-07-02,2022-02-07 16:50:18,2022-02-07 16:53:17,нет регистрации,в комнате,3
2,258106,1205697,А,включён,0,2022-07-02,2022-02-07 16:50:18,2022-02-07 16:53:17,нет регистрации,нет в комнате,3
3,258106,1205697,В,включён,0,2022-07-02,2022-02-07 16:50:18,2022-02-07 16:53:17,нет регистрации,в комнате,3
4,258109,1206994,А,выключен,1,2022-02-08,2022-02-08 03:05:31,2022-02-08 03:06:58,нет регистрации,нет в комнате,1
5,258109,1206994,А,выключен,0,2022-02-08,2022-02-08 03:05:31,2022-02-08 03:06:58,нет регистрации,нет в комнате,1
6,258110,1208135,А,выключен,2,2022-09-02,2022-02-09 14:31:41,2022-02-09 14:33:41,нет регистрации,нет в комнате,2
7,258110,1208135,А,включён,1,2022-09-02,2022-02-09 14:31:41,2022-02-09 14:33:41,нет регистрации,нет в комнате,2
8,258110,1208135,Б,выключен,2,2022-09-02,2022-02-09 14:31:41,2022-02-09 14:33:41,нет регистрации,нет в комнате,2
9,258110,1208135,Б,включён,1,2022-09-02,2022-02-09 14:31:41,2022-02-09 14:33:41,нет регистрации,в комнате,2


Кроме того пропусков или некорректых данных в данной таблице не обнаружено.

In [195]:
opros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   code                       207 non-null    int64         
 1   passport                   207 non-null    int64         
 2   button                     207 non-null    object        
 3   tv state                   207 non-null    object        
 4   unit                       207 non-null    int64         
 5   day                        207 non-null    datetime64[ns]
 6   cs_start                   207 non-null    datetime64[ns]
 7   cs_finish                  207 non-null    datetime64[ns]
 8   see                        207 non-null    object        
 9   display                    207 non-null    object        
 10  Кол-во человек в семье 4+  207 non-null    int64         
dtypes: datetime64[ns](3), int64(4), object(4)
memory usage: 17.9+ KB


### Приборы

Далее взглянем на данные с приборов.

In [196]:
pipl.head(10)

,PassportNumber,UnitNumber,DeviceState,Begin,End
0,10100003,0,OFF,2022-02-07 02:00:00.000,2022-02-08 02:00:00.000
1,10100003,0,OFF,2022-02-08 02:00:00.000,2022-02-08 11:48:42.000
2,10100003,0,ON,2022-02-08 11:48:42.000,2022-02-08 12:36:55.000
3,10100003,0,OFF,2022-02-08 12:36:55.000,2022-02-08 22:05:17.000
4,10100003,0,ON,2022-02-08 22:05:17.000,2022-02-08 23:48:03.000
5,10100003,0,OFF,2022-02-08 23:48:03.000,2022-02-09 02:00:00.000
6,10100003,0,OFF,2022-02-09 02:00:00.000,2022-02-10 02:00:00.000
7,10100003,0,OFF,2022-02-10 02:00:00.000,2022-02-10 19:16:30.000
8,10100003,0,ON,2022-02-10 19:16:30.000,2022-02-10 21:24:27.000
9,10100003,0,OFF,2022-02-10 21:24:27.000,2022-02-11 02:00:00.000


Как и в случае с опросными данными переведём время в удобный формат.

In [197]:
pipl['Begin'] = pd.to_datetime(pipl['Begin'])
pipl['End'] = pd.to_datetime(pipl['End'])

pipl.head(10)

,PassportNumber,UnitNumber,DeviceState,Begin,End
0,10100003,0,OFF,2022-02-07 02:00:00,2022-02-08 02:00:00
1,10100003,0,OFF,2022-02-08 02:00:00,2022-02-08 11:48:42
2,10100003,0,ON,2022-02-08 11:48:42,2022-02-08 12:36:55
3,10100003,0,OFF,2022-02-08 12:36:55,2022-02-08 22:05:17
4,10100003,0,ON,2022-02-08 22:05:17,2022-02-08 23:48:03
5,10100003,0,OFF,2022-02-08 23:48:03,2022-02-09 02:00:00
6,10100003,0,OFF,2022-02-09 02:00:00,2022-02-10 02:00:00
7,10100003,0,OFF,2022-02-10 02:00:00,2022-02-10 19:16:30
8,10100003,0,ON,2022-02-10 19:16:30,2022-02-10 21:24:27
9,10100003,0,OFF,2022-02-10 21:24:27,2022-02-11 02:00:00


Кроме того пробелов и других некорректностей в данных не обнаружено.

In [198]:
pipl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642134 entries, 0 to 642133
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PassportNumber  642134 non-null  int64         
 1   UnitNumber      642134 non-null  int64         
 2   DeviceState     642134 non-null  object        
 3   Begin           642134 non-null  datetime64[ns]
 4   End             642134 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 24.5+ MB


## Анализ

### Ложноположительные ответы

Для того чтобы ответить на вопрос о количестве ложноположительных ответов соотнесём адрес телевизора, который определяется паспортом и идентификатором и время его включения согласно опросам с фактическим включением устройства. Найдём количество несоответсвий.

In [199]:
lp1 = 0
nlp1 = len(opros.loc[opros['tv state'] == 'включён'])

for passport in opros.loc[opros['tv state'] == 'включён', 'passport'].unique():
  for unit in opros.loc[(opros['tv state'] == 'включён') & \
                        (opros['passport'] == passport), 'unit'].unique():
    start = opros.loc[(opros['tv state'] == 'включён') & \
                      (opros['passport'] == passport) & \
                      (opros['unit'] == unit), 'cs_start'].sort_values(ascending=True).reset_index(drop=True)
    finish = opros.loc[(opros['tv state'] == 'включён') & \
                       (opros['passport'] == passport) & \
                       (opros['unit'] == unit), 'cs_finish'].sort_values(ascending=True).reset_index(drop=True)
    # если их количество различно?
    if start.loc[0] < finish.loc[0]: # нужно, если в начале эксперемента не все телевизоры были выключены
      z = 0
    else:
      z = 1 # если это всё ещё меньше чем старт?
    #print(start, finish)
    for i in range(0,len(start) - z):
      if ((pipl.loc[(pipl['DeviceState'] == 'OFF') & \
          (pipl['PassportNumber'] == passport) & \
          (pipl['UnitNumber'] == unit), 'Begin'] < start.loc[i]) & \
          (pipl.loc[(pipl['DeviceState'] == 'OFF') & (pipl['PassportNumber'] == passport) & \
          (pipl['UnitNumber'] == unit), 'End'] > finish.loc[i + z])).sum():
        lp1 = lp1 + 1
      #print(pipl.loc[(pipl['DeviceState'] == 'OFF') & \
      #      (pipl['PassportNumber'] == passport) & \
      #      (pipl['UnitNumber'] == unit), ['Begin', 'End']])

print("Количество положительных ответов:", nlp1)
print("Количество ложноположительных ответов:", lp1)
print("Доля ложноположительных ответов:", lp1 / nlp1)

Количество положительных ответов: 54
Количество ложноположительных ответов: 0
Доля ложноположительных ответов: 0.0


### Ложноотрицательные ответы

Для того чтобы ответить на вопрос о количестве ложноположительных ответов соотнесём адрес телевизора, который определяется паспортом и идентификатором и время его неактивности согласно опросам с фактическим временем неактивности устройства. Найдём количество несоответсвий.

In [202]:
lo1 = 0
nlo1 = len(opros.loc[opros['tv state'] == 'выключен'])

for passport in opros.loc[opros['tv state'] == 'выключен', 'passport'].unique():
  for unit in opros.loc[(opros['tv state'] == 'выключен') & \
                        (opros['passport'] == passport), 'unit'].unique():
    start = opros.loc[(opros['tv state'] == 'выключен') & \
                      (opros['passport'] == passport) & \
                      (opros['unit'] == unit), 'cs_start'].sort_values(ascending=True).reset_index(drop=True)
    finish = opros.loc[(opros['tv state'] == 'выключен') & \
                       (opros['passport'] == passport) & \
                       (opros['unit'] == unit), 'cs_finish'].sort_values(ascending=True).reset_index(drop=True)
    # если их количество различно?
    if start.loc[0] < finish.loc[0]: # нужно, если в начале эксперемента не все телевизоры были выключены
      z = 0
    else:
      z = 1 # если это всё ещё меньше чем старт?
    #print(start, finish)
    for i in range(0,len(start) - z):
      if ((pipl.loc[(pipl['DeviceState'] == 'ON') & \
          (pipl['PassportNumber'] == passport) & \
          (pipl['UnitNumber'] == unit), 'Begin'] < start.loc[i]) & \
          (pipl.loc[(pipl['DeviceState'] == 'ON') & (pipl['PassportNumber'] == passport) & \
          (pipl['UnitNumber'] == unit), 'End'] > finish.loc[i + z])).sum():
        lo1 = lo1 + 1
      #print(pipl.loc[(pipl['DeviceState'] == 'ON') & \
      #      (pipl['PassportNumber'] == passport) & \
      #      (pipl['UnitNumber'] == unit), ['Begin', 'End']])

print("Количество ложноотрицательных ответов:", nlo1)
print("Количество отрицательных ответов:", lo1)
print("Доля ложноотрицательных ответов:", lo1 / nlo1)

Количество ложноотрицательных ответов: 153
Количество отрицательных ответов: 3
Доля ложноотрицательных ответов: 0.0196078431372549


### Заочная регистрация

Выделим ситуации при которых респондент, не имеющий регистрации, находился в комнате с включённым телевизором, при этом у других респондентов в домохозяйстве имелась регистрация.

In [230]:
ctv = opros.loc[(opros['Кол-во человек в семье 4+'] > 1) & \
                (opros['display'] == 'в комнате') & \
                (opros['tv state'] == 'включён') & \
                (opros['see'] == 'нет регистрации')]

atv = {}
for passport in ctv['passport'].unique():
  for unit in ctv.loc[ctv['passport'] == passport, 'unit'].unique():
    for button in ctv.loc[(ctv['passport'] == passport) & \
     (ctv['unit'] == unit), 'button'].unique():
      if (passport, unit) in atv:
        atv[passport, unit] = np.append(atv[passport, unit], button)
      else:
        atv[passport, unit] = np.array(button)

i = []
for passport,unit in atv:
  i = list(i + (opros[(opros['passport'] == passport) & \
               (opros['unit'] == unit) & \
               (opros['see'] == 'есть регистрация')].index.tolist()))
b = opros.loc[i]

j = []
for passport,unit in atv:
  for button in b.loc[(b['passport'] == passport) & (b['unit'] == unit), 'button'].unique():
    j = list(j + (opros[(opros['passport'] == passport) & \
                        (opros['unit'] == unit) & \
                        (opros['button'] != button)].index.tolist()))
a = opros.loc[j]



---
При этом можно выделить ситуации отсутствия регистрации у людей, имеющих дома других людей с регистрацией:


In [231]:
opros.loc[j].sort_values(by=['Кол-во человек в семье 4+']).set_index('Кол-во человек в семье 4+')

,code,passport,button,tv state,unit,day,cs_start,cs_finish,see,display
Кол-во человек в семье 4+,,,,,,,,,,
2,258124,1210425,Б,включён,0,2022-07-02,2022-02-07 17:57:43,2022-02-07 18:07:25,нет регистрации,в комнате
4,258128,1210930,Б,включён,1,2022-07-02,2022-02-07 18:52:47,2022-02-07 18:54:12,нет регистрации,в комнате
4,258128,1210930,В,включён,1,2022-07-02,2022-02-07 18:52:47,2022-02-07 18:54:12,нет регистрации,нет в комнате
4,258128,1210930,Г,включён,1,2022-07-02,2022-02-07 18:52:47,2022-02-07 18:54:12,нет регистрации,нет в комнате


Когда были следующие люди с регистрацией:

In [232]:
opros.loc[i].sort_values(by=['Кол-во человек в семье 4+']).set_index('Кол-во человек в семье 4+')

,code,passport,button,tv state,unit,day,cs_start,cs_finish,see,display
Кол-во человек в семье 4+,,,,,,,,,,
2,258124,1210425,А,включён,0,2022-07-02,2022-02-07 17:57:43,2022-02-07 18:07:25,есть регистрация,в комнате
4,258128,1210930,А,включён,1,2022-07-02,2022-02-07 18:52:47,2022-02-07 18:54:12,есть регистрация,в комнате


## Вывод

В данной работе была произведена обработка результатов исследования, целью которого является проверка работы оборудования и регистрации респондентов на пиплметре (измерительное устройство для фиксации телесмотрения).

В ходе анализа было установлено, что данные имеют нулевой процент ложноположительных исследований и ~2% ложноотрицательных результатов. Кроме того были выделены спецефические ситуации заочной регистрации респондетами других респондентов.